In [1]:
import pandas as pd
from sampling import *
from sklearn.preprocessing import StandardScaler
import preprocess_data
from autogluon.tabular import TabularDataset, TabularPredictor

2025-03-23 19:42:01.325492: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742755321.338058   11545 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742755321.341768   11545 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742755321.352315   11545 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742755321.352336   11545 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742755321.352338   11545 computation_placer.cc:177] computation placer alr

In [2]:
# Load your dataset
df = pd.read_csv('task/train_set/x_train_aggregated.csv') # Replace with your file path
x = df.drop(columns=['AccountID'])
y_df = pd.read_csv('task/train_set/y_train.csv')
# df['Fraudster'] = y_df['Fraudster']
xy_train = pd.merge(df, y_df, on='AccountID', how='inner')

df_val = pd.read_csv('task/validation_set/x_val_aggregated.csv')
x_val = df_val.drop(columns=['AccountID'])
yval_df = pd.read_csv('task/validation_set/y_val.csv')
# df_val['Fraudster'] = yval_df['Fraudster']
xy_val = pd.merge(df_val, yval_df, on='AccountID', how='inner')
# y_val = yval_df['Fraudster']
# Display the first few rows
# print(df.head())

test_df = pd.read_csv('task/test_set/x_test.csv')
test_agg_df = pd.read_csv('task/test_set/x_test_aggregated.csv')

# Check the distribution of the target variable (Fraudster)
# print(df_val['Fraudster'].value_counts())

xtrain_df = pd.read_csv('task/train_set/x_train.csv')
x_val_df = pd.read_csv('task/validation_set/x_val.csv')

In [3]:
df_fea, df_val_fea, df_test_fea = preprocess_data.preprocess_and_extract_features()


df_train_fea shape: (11061, 25)
df_val_fea shape: (11060, 25)
df_test_fea shape: (11060, 24)


In [4]:
print("Training Data:", df_fea.head())
# print("Validation Data:", df_val_fea.head())
# print("Test Data:", df_test_fea.head())

Training Data:      AccountID  NumTransactions      AvgAmount   MaxAmount      TotalIn  \
0  C0001143040              562  146658.532989  3585064.01  35124031.74   
1  C0001332727              178  128888.997753   903276.53  13396604.25   
2  C0002641069               56  146342.356786  2571512.49   4547603.60   
3  C0002967890              103  117303.432330  3554689.54   3016367.57   
4  C0003513711               50  144333.040800  1563406.66   4462829.33   

       MaxIn     TotalOut  MaxCashOut  MaxDebit  MaxTransfer  ...  \
0  424498.32  47298063.80   284531.00   8893.26   3585064.01  ...   
1  341238.53   9545637.35   258056.33      0.00    903276.53  ...   
2  293992.40   3647568.38   204095.40   3594.16   2571512.49  ...   
3  392633.82   9065885.96   247445.60  23100.20   3554689.54  ...   
4  264727.14   2753822.71   142728.98   3080.34   1563406.66  ...   

   BalanceDeviation_mean  TransactionAmountDeviation_max  \
0          -2.038304e-10                    3.438405e+06   

In [5]:
# from sklearn.feature_selection import mutual_info_classif
# import matplotlib.pyplot as plt

# # df_fe3 = pd.merge(df_fe, df, on='AccountID', how='inner')
# X = df_fea.drop(columns=['AccountID','Fraudster'])
# y = df_fea['Fraudster']

# x_val = df_val_fea.drop(columns=['AccountID','Fraudster'])
# y_val = df_val_fea['Fraudster']

# X_test = df_test_fea.drop(columns=['AccountID'])

# # Calculate Mutual Information scores
# mi_scores = mutual_info_classif(X, y)

# # Create a DataFrame to store feature names and their MI scores
# mi_df = pd.DataFrame({'Feature': X.columns, 'MI_Score': mi_scores})
# mi_df = mi_df.sort_values(by='MI_Score', ascending=False)

# # Plot the MI scores
# plt.figure(figsize=(10, 6))
# plt.barh(mi_df['Feature'], mi_df['MI_Score'])
# plt.xlabel('Mutual Information Score')
# plt.title('Feature Importance (Mutual Information)')
# plt.show()

# # Select top k features
# k = 10 # Number of top features to select
# top_features_mi = mi_df['Feature'][:k].tolist()
# print("Top Features (Mutual Information):", top_features_mi)

In [6]:
# # Combine selected features from all methods
# selected_features = list(set(top_features_mi ))
# # selected_features = list(set(top_features_mi).intersection(selected_features_l1))
# print("Final Selected Features:", selected_features)

In [7]:
# df_fea = df_fea[selected_features + ['AccountID', 'Fraudster']]
# df_val_fea = df_val_fea[selected_features+ ['AccountID', 'Fraudster']]
# df_test_fea = df_test_fea[selected_features+ ['AccountID']]

In [8]:
df_fea.head()

,AccountID,NumTransactions,AvgAmount,MaxAmount,TotalIn,MaxIn,TotalOut,MaxCashOut,MaxDebit,MaxTransfer,...,BalanceDeviation_mean,TransactionAmountDeviation_max,TransactionFrequencyPerHour_max,TransactionBurstiness_mean,IsUnusualTransactionType_sum,IsExternalAccount_max,ExternalAccountFrequency_mean,isUnauthorizedOverdraft_sum,RollingTransactionCount_mean,Fraudster
0,C0001143040,562,146658.532989,3585064.01,35124031.74,424498.32,47298063.80,284531.00,8893.26,3585064.01,...,-2.038304e-10,3.438405e+06,12,211.339858,4,1,1252.549451,0,9.919929,0
1,C0001332727,178,128888.997753,903276.53,13396604.25,341238.53,9545637.35,258056.33,0.00,903276.53,...,-2.262905e-10,7.743875e+05,6,40.780899,0,1,14.119048,0,9.747191,0
2,C0002641069,56,146342.356786,2571512.49,4547603.60,293992.40,3647568.38,204095.40,3594.16,2571512.49,...,1.372038e-10,2.425170e+06,2,1.946429,1,1,4529.437500,0,9.196429,1
3,C0002967890,103,117303.432330,3554689.54,3016367.57,392633.82,9065885.96,247445.60,23100.20,3554689.54,...,-1.537134e-10,3.437386e+06,4,17.271845,0,1,2224.230769,0,9.563107,0
4,C0003513711,50,144333.040800,1563406.66,4462829.33,264727.14,2753822.71,142728.98,3080.34,1563406.66,...,8.381903e-11,1.419074e+06,2,4.040000,3,1,701.142857,0,9.100000,0


In [9]:
# fraud_detector = FraudDetection(df_fea,df_val_fea)
# fraud_detector.preprocess_data()
# scaler = StandardScaler()

# # Generate oversampled data
# X_train_smote, y_train_smote = fraud_detector.oversample_data()
# X_scaled = scaler.fit_transform(x_val)
# x_val = pd.DataFrame(scaler.transform(x_val), columns=x_val.columns)

# X_train_smote


In [10]:
# X_train_smote_df = pd.DataFrame(X_train_smote, columns=selected_features)

In [11]:
# X_train_smote['Fraudster'] = y_train_smote

In [12]:
df_fea.to_csv('task/autogluon/x_train.csv', index=False)
df_val_fea.to_csv('task/autogluon/x_val.csv', index=False)
df_test_fea.to_csv('task/autogluon/x_test.csv', index=False)

In [13]:
train_data = TabularDataset('task/autogluon/x_train.csv')
train_data.head()

label = 'Fraudster'
problem_type = 'binary'
eval_metric = 'f1'
#print(train_data[label].describe())
predictor = TabularPredictor(label=label, problem_type=problem_type, eval_metric=eval_metric ).fit(train_data)

test_data = TabularDataset('task/autogluon/x_val.csv')

y_pred = predictor.predict(test_data.drop(columns=[label]))

predictor.evaluate(test_data, silent=True)

No path specified. Models will be saved in: "AutogluonModels/ag-20250323_184253"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.12.0
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #53~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Wed Jan 15 19:18:46 UTC 2
CPU Count:          16
Memory Avail:       6.72 GB / 14.87 GB (45.2%)
Disk Space Avail:   17.13 GB / 176.11 GB (9.7%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in co

[1000]	valid_set's binary_logloss: 0.274881	valid_set's f1: 0.532374


	0.5382	 = Validation score   (f1)
	2.22s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: LightGBM ...
	0.5694	 = Validation score   (f1)
	1.53s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.4464	 = Validation score   (f1)
	0.88s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.4329	 = Validation score   (f1)
	0.92s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: CatBoost ...
	0.5116	 = Validation score   (f1)
	1.59s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.25	 = Validation score   (f1)
	0.47s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.2673	 = Validation score   (f1)
	0.43s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
No improvement since epoch 1: early stopping
	0.5038	 = Validation score   (f1)
	4.8s	 = Training   runtime
	0.01s	 = Vali

{'f1': 0.5634718032025993,
 'accuracy': 0.8299276672694395,
 'balanced_accuracy': 0.7907433630231754,
 'mcc': 0.4851550118199895,
 'roc_auc': 0.9045324621979542,
 'precision': 0.45690628528415506,
 'recall': 0.7348668280871671}

In [14]:
print(predictor.leaderboard(test_data))

/home/zoro/anaconda3/envs/bbdc/lib/python3.12/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


                  model  score_test  score_val eval_metric  pred_time_test  \
0               XGBoost    0.569231   0.546125          f1        0.023879   
1              CatBoost    0.567095   0.511628          f1        0.005148   
2              LightGBM    0.564895   0.569395          f1        0.050070   
3   WeightedEnsemble_L2    0.563472   0.600000          f1        0.328680   
4        NeuralNetTorch    0.563394   0.574257          f1        0.049391   
5            LightGBMXT    0.554035   0.538182          f1        0.100059   
6         LightGBMLarge    0.544581   0.516854          f1        0.017991   
7      RandomForestEntr    0.521092   0.432900          f1        0.076882   
8      RandomForestGini    0.517106   0.446352          f1        0.098409   
9       NeuralNetFastAI    0.482289   0.503817          f1        0.065902   
10       ExtraTreesEntr    0.426994   0.267327          f1        0.106097   
11       ExtraTreesGini    0.416804   0.250000          f1      

In [43]:
y_pred = predictor.predict(test_data.drop(columns=[label]))
print(len(y_pred))  

11060


/home/zoro/anaconda3/envs/bbdc/lib/python3.12/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


In [51]:
xy_pred = pd.DataFrame(y_pred)
xy_pred['AccountID'] = df_val_fea['AccountID'].values
xy_pred_merge = pd.merge(yval_df, xy_pred, on='AccountID', how='inner')
xy_pred_merge.to_csv('task/autogluon/y_val_model_pred.csv', index=False)

y_true = xy_pred_merge['Fraudster_x']
y_predy = xy_pred_merge['Fraudster_y']

In [52]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
print("Confusion Matrix:\n", confusion_matrix(y_true, y_predy))
print("\nClassification Report:\n", classification_report(y_true, y_predy))


Confusion Matrix:
 [[7965 1443]
 [ 438 1214]]

Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.85      0.89      9408
           1       0.46      0.73      0.56      1652

    accuracy                           0.83     11060
   macro avg       0.70      0.79      0.73     11060
weighted avg       0.87      0.83      0.84     11060



In [53]:
#predicting over x_test
test_data_up = TabularDataset('task/autogluon/x_test.csv')
y_test_pred = predictor.predict(test_data_up)
print(len(y_test_pred))


Loaded data from: task/autogluon/x_test.csv | Columns = 24 / 24 | Rows = 11060 -> 11060
/home/zoro/anaconda3/envs/bbdc/lib/python3.12/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


11060


In [54]:
xy_test_pred = pd.DataFrame(y_test_pred)
xy_test_pred['AccountID'] = df_test_fea['AccountID'].values
xskel = pd.read_csv('task/student_skeleton.csv')
xskel = xskel[['AccountID']]
xy_test_pred_merge = pd.merge(xskel, xy_test_pred, on='AccountID', how='inner')
xy_test_pred_merge.to_csv('submission_files/Submission_5.csv', index=False)
xy_test_pred_merge.head()

,AccountID,Fraudster
0,C3393649433,0
1,C0855769514,0
2,C3140924771,0
3,C5919965464,0
4,C1039024463,0


In [55]:
xy_test_pred_merge['Fraudster'].value_counts()

Fraudster
0    8552
1    2508
Name: count, dtype: int64

In [ ]:
predictor.feature_importance(test_data)

These features in provided data are not utilized by the predictor and will be ignored: ['AccountID']
Computing feature importance via permutation shuffling for 23 features using 5000 rows with 5 shuffle sets...
/home/zoro/anaconda3/envs/bbdc/lib/python3.12/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")
	24.29s	= Expected runtime (4.86s per shuffle set)
/home/zoro/anaconda3/envs/bbdc/lib/python3.12/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute

,importance,stddev,p_value,n,p99_high,p99_low
TotalOut,0.242276,0.016900,0.000003,5,0.277075,0.207478
TotalIn,0.235500,0.017243,0.000003,5,0.271004,0.199996
ExternalAccountFrequency_mean,0.178436,0.015858,0.000007,5,0.211089,0.145784
TimeSinceLastTransaction_mean,0.104389,0.006952,0.000002,5,0.118703,0.090075
MaxDebit,0.067848,0.011545,0.000097,5,0.091619,0.044076
IsUnusualTransactionType_sum,0.050486,0.007746,0.000064,5,0.066435,0.034538
AvgAmount,0.043984,0.009902,0.000288,5,0.064372,0.023596
RollingTransactionCount_mean,0.029485,0.010242,0.001498,5,0.050574,0.008396
TransactionFrequencyPerHour_max,0.029429,0.008743,0.000834,5,0.047431,0.011427
MaxFreqH,0.023016,0.008729,0.002070,5,0.040988,0.005043
